In [ ]:
%matplotlib inline
#
from lrgsglib import *
#
move_to_rootf(pathname='lrg_eegfc')
#
from lrg_eegfc import *

### data reader

In [ ]:
data_dict, int_label_pos_map = load_data_dict(
    mat_path, patients_list, phase_labels, param_keys_list
)

### single phase

In [ ]:
patient = 'Pat_02'
phase = 'rsPre'
#
corr_net_filt = dict(threshold=0)
#
data_pat_phase = data_dict[patient][phase]
data_pat_phase_ts = data_pat_phase['data']
fs = data_pat_phase['fs']
filter_order = data_pat_phase['filter_order']
#
n_intervals = 10
interval_length = int(data_pat_phase_ts.shape[1] // n_intervals)
n_intervals, interval_length

In [ ]:
# Use the function from corrmat module to build correlation matrices per band
corr_mat_band_tw = []
for i in range(n_intervals):
    start = i * interval_length
    end = (i + 1) * interval_length
    interval_data = data_pat_phase_ts[:, start:end]
    corr_mat_band_tw.append(
        build_corrmat_perband(
            interval_data,
            fs,
            apply_threshold_filtering=True,
        )
    )
    clear_output()

In [ ]:
def plot_comprehensive_analysis(corr_mat, band_name, phase, interval_idx, all_labels, patient):
    """
    Plot 4-panel analysis: network, dendrogram, correlation matrix, and specific heat.
    """
    # Create network and filter
    G = nx.from_numpy_array(corr_mat)
    G.remove_edges_from([(u, v) for u, v, d in G.edges(data=True) if d['weight'] == 0])
    G.remove_nodes_from(list(nx.isolates(G)))
    
    if len(G.nodes()) == 0:
        print(f"No nodes remaining for {band_name}, {phase}, interval {interval_idx}")
        return
    
    # Get giant component
    G_giant, removed_nodes = get_giant_component_leftoff(G)
    
    if len(G_giant.nodes()) == 0:
        print(f"No giant component for {band_name}, {phase}, interval {interval_idx}")
        return
    
    # Create filtered labels dictionary
    labeldict = {k: v for k, v in all_labels.to_dict().items() if k in G_giant.nodes()}
    
    # Compute hierarchical clustering and specific heat
    spect, L, rho, Trho, tau = compute_laplacian_properties(G_giant, tau=None)
    dists = squareform(Trho)
    lnkgM, label_list, _ = compute_normalized_linkage(dists, G_giant, method='ward')
    clTh, *_ = compute_optimal_threshold(lnkgM, scaling_factor=0.98)
    
    # Compute specific heat (entropy derivative)
    _, dS, _, t_values = entropy(G_giant)
    
    # Create 2x2 subplot
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Network with Kamada-Kawai layout (top-left)
    node_list = list(G_giant.nodes())
    labels_for_dendro = [labeldict[n] for n in node_list]
    
    # First create dendrogram to get colors
    temp_dendro = dendrogram(lnkgM, color_threshold=clTh, labels=labels_for_dendro, 
                            above_threshold_color='k', no_plot=True)
    leaf_label_colors = {lbl: col for lbl, col in zip(temp_dendro['ivl'], temp_dendro['leaves_color_list'])}
    
    # Plot network
    widths = [G_giant[u][v]['weight'] for u, v in G_giant.edges()]
    edge_colors = ['red' if d['weight'] < 0 else 'blue' for u, v, d in G_giant.edges(data=True)]
    node_colors = [leaf_label_colors.get(labeldict[n], 'gray') for n in node_list]
    
    pos = nx.kamada_kawai_layout(G_giant)
    nx.draw(G_giant, ax=axes[0,0], pos=pos, width=widths, edge_color=edge_colors,
            labels=labeldict, node_color=node_colors, node_size=100, font_size=8, with_labels=True)
    axes[0,0].set_title(f'Network - {band_name}\nNodes: {len(G_giant.nodes())}')
    
    # 2. Dendrogram with optimal cut (top-right)
    dendro = dendrogram(lnkgM, ax=axes[0,1], color_threshold=clTh, labels=labels_for_dendro,
                       above_threshold_color='k', leaf_font_size=6, orientation='top')
    
    tmin, tmax = lnkgM[:, 2][0] * 0.8, lnkgM[:, 2][-1] * 1.01
    axes[0,1].set_yscale('log')
    axes[0,1].axhline(clTh, color='b', linestyle='--', alpha=0.7, label=f'Cut: {clTh:.3f}')
    axes[0,1].set_ylim(tmin, tmax)
    axes[0,1].set_ylabel(r'$\mathcal{D}/\mathcal{D}_{max}$')
    axes[0,1].set_title(f'Dendrogram - {band_name}')
    axes[0,1].tick_params(axis='x', labelsize=6)
    axes[0,1].legend()
    
    # 3. Correlation matrix (bottom-left)
    im = axes[1,0].imshow(corr_mat, cmap='RdBu_r', vmin=-1, vmax=1, interpolation='nearest')
    axes[1,0].set_title(f'Correlation Matrix - {band_name}')
    axes[1,0].set_xlabel('Node Index')
    axes[1,0].set_ylabel('Node Index')
    plt.colorbar(im, ax=axes[1,0], fraction=0.046, pad=0.04)
    
    # 4. Specific Heat curve (bottom-right)
    if len(dS) > 1 and len(t_values) > 1:
        axes[1,1].plot(t_values[1:], dS, 'o-', linewidth=2, markersize=4)
        axes[1,1].set_xscale('log')
        axes[1,1].set_xlabel(r'$\tau$')
        axes[1,1].set_ylabel('Specific Heat (dS)')
        axes[1,1].set_title(f'Specific Heat - {band_name}')
        axes[1,1].grid(True, alpha=0.3)
        
        # Mark maximum if exists
        if len(dS) > 0:
            max_idx = np.argmax(dS)
            if max_idx < len(t_values[1:]):
                axes[1,1].axvline(t_values[1:][max_idx], color='red', linestyle='--', alpha=0.7, 
                                 label=f'Max: τ={t_values[1:][max_idx]:.2f}')
                axes[1,1].legend()
    else:
        axes[1,1].text(0.5, 0.5, 'Insufficient data\nfor specific heat', 
                      ha='center', va='center', transform=axes[1,1].transAxes)
        axes[1,1].set_title(f'Specific Heat - {band_name}')
    
    # Overall title
    fig.suptitle(f'{patient} - {phase} - Interval {interval_idx} - {band_name} band', fontsize=16)
    plt.tight_layout()
    
    return fig

In [ ]:
# Main visualization loop
save_figs = True  # Set to False if you don't want to save figures
fig_path = Path('data') / 'figures' / 'time_windows_analysis' / patient / phase
if save_figs:
    fig_path.mkdir(parents=True, exist_ok=True)

# Process per band, then per time interval
for band_name in BRAIN_BANDS.keys():
    print(f"\nProcessing band: {band_name}")
    for interval_idx in range(n_intervals):
        print(f"  Interval {interval_idx+1}/{n_intervals}")

        # correlation matrices for this interval
        interval_corr_mats = corr_mat_band_tw[interval_idx]

        # if band present for this interval
        if band_name in interval_corr_mats:
            corr_mat = interval_corr_mats[band_name]

            fig = plot_comprehensive_analysis(
                corr_mat, band_name, phase, interval_idx,
                int_label_pos_map[patient]['label'], patient
            )

            if fig is not None:
                if save_figs:
                    filename = f'{patient}_{phase}_{band_name}_interval_{interval_idx:02d}.pdf'
                    fig.savefig(fig_path / filename, bbox_inches='tight', dpi=300)
                    print(f"    Saved: {filename}")

                # plt.show()
                plt.close(fig)
        else:
            print(f"    No data available for {band_name} at interval {interval_idx}")

print(f"\nCompleted visualization for {patient} - {phase}")
print(f"Processed {len(BRAIN_BANDS)} bands × {n_intervals} intervals = {len(BRAIN_BANDS) * n_intervals} total plots")

In [ ]:
corr_mat_band_tw['delta'][interval_idx]['delta']